In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
!pip install imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv('data_dummies.csv')
df

In [ ]:
df_regression = df[['indicatrice__Cancers', 'classe_age']].dropna()

# Extract the independent variable (X) and dependent variable (y)
X = df_regression[['classe_age']]
y = df_regression['indicatrice__Cancers']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Resample the training data to address class imbalance
rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# Apply SMOTE to the resampled data
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_resampled, y_resampled)

# Create a logistic regression model with balanced class weights
model = LogisticRegression(class_weight='balanced')

# Fit the model to the resampled training data
model.fit(X_resampled, y_resampled)

# Make predictions on the testing data
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
classification_rep = classification_report(y_test, predictions, zero_division=1)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

In [ ]:

# Plot the logistic regression curve
X_values = np.linspace(X.min(), X.max(), 1000).reshape(-1, 1)
probabilities = model.predict_proba(X_values)[:, 1]

plt.scatter(X, y, label='Original data')
plt.plot(X_values, probabilities, color='red', label='Logistic regression curve')
plt.xlabel('Age')
plt.ylabel('Probability of Cancer')
plt.title('Logistic Regression of Cancer on Age')
plt.legend()
plt.show()

In [ ]:
#regression logisitique qu'il faudra interpreter à un moment